# Profitable Apps in Google Play & App Store Markets

The goal of this project is to determine what makes an app profitable on both the Google Play and App Store markets. This would allow a company to make decisions on which apps to build based on data. 

For the sake of this project, the assumption is that all apps that will be made are free, for English-speaking users, and depend heavily on in-app ads. As a result, the number of users heavily influences the profitability of the app. 

## Opening the Data

In order to achieve this goal, I reviewed data from two sources:
- [Google Play](https://www.kaggle.com/datasets/lava18/google-play-store-apps): collected in August 2018 and containing approximately 10,000 apps
- [App Store](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps): collected in July 2017 and containing approximately 7,000 apps

In [1]:
from csv import reader

#Google Play
read_file = reader(open('googleplaystore.csv'))
android_data = list(read_file)
android_header = android_data[0]
android = android_data[1:]

#App Store
read_file = reader(open('AppleStore.csv'))
ios_data = list(read_file)
ios_header = ios_data[0]
ios = ios_data[1:]

## Exploring the Data

The function below will help in exploring the data by slicing the data to only the rows that we we interested in, while also printing them in a more readable way. Keep in mind that the `end` parameter is not inclusive.

If the row_and_columns argument is `True`, then it will also let us know how many rows (apps) and columns (data points for each app) are in the data.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    for row in dataset[start:end]:
        print(row)
        print('\n')
    
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Let's take a look at the Android Data column names, along with some random rows of data. We will also determine how many rows and columns there are in this set. 

In [3]:
print(android_header)
print('\n')
print(explore_data(android, 0, 5, True))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000', 'Free', '0', 'Everyone

We can quickly see that the Android data had 10,841 apps with 13 columns. At a glance, I would think that `App`, `Category`, `Reviews`, `Installs`, `Type`, `Price`, and `Genres` will be useful.

Now, let's do the same for the iOS data:

In [4]:
print(ios_header)
print('\n')
print(explore_data(ios, 0, 5, True))

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16
None


The iOS data consists of 7,197 apps and has 16 columns. At a glance, `track_name`, `currency`, `price`, `rating_count_tot`, `rating_count_ver` and `prime_genre` will be useful. 

_Note_: These column names are a bit harder to understand. You can find their descriptions in the data documentation link found in the 'Opening the Data' section.

## Data Cleaning

The Google Play discussion section includes [a thread](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) for an error on row 10472 (if using full data set with header, it will be row 10473).

In [6]:
for i in range(len(android[0])):
    print (f'{android_header[i]}: {android[10472][i]} ')

App: Life Made WI-Fi Touchscreen Photo Frame 
Category: 1.9 
Rating: 19 
Reviews: 3.0M 
Size: 1,000 
Installs: Free 
Type: 0 
Price: Everyone 
Content Rating:  
Genres: February 11, 2018 
Last Updated: 1.0.19 
Current Ver: 4.0 and up 
Android Ver:  


Looking at this, we can definitely see that something is off. Upon further exploration, it seems that the `Category` and `Genres` are missing, which is causing all other data to get moved. 

There are several ways of handling this, including deleting the row outright or finding the missing information, adding it into the set, and moving the other information to this correct place. For the sake of this project, I will delete it outright.

In [7]:
del android[10472]

The previous number of rows was 10,841. If the below prints as one less, the delete was successful.

In [8]:
print(len(android))

10840


# Removing Duplicate Entries

The function below can be called on either data set to determine if and how many potential duplicates exist in the data set.

In [9]:
def duplicates(dataset):
    duplicate_apps = []
    unique_apps = []
    
    for app in dataset:
        if app[0] not in unique_apps:
            unique_apps.append(app[0])
        else:
            duplicate_apps.append(app[0])
    
    return duplicate_apps

android_duplicates = duplicates(android)
ios_duplicates = duplicates(ios)

### Android Data

I will look at the Android data first.

In [23]:
print('Number of duplicate apps:', len(android_duplicates))

Number of duplicate apps: 1181


In [24]:
print('Examples of duplicate apps:', android_duplicates[:15])

Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


I will be picking a couple of random duplicate names and will print them out to see if I can find any differences with them. This will help me come up with criterion to use for determining which duplicate to delete.

To make this easier, I will create a function that I can call for each dataset and name I want to check.

In [12]:
def specific_duplicates(dataset, name):
    for app in dataset:
        if app[0] == name:
            print(app)
            
specific_duplicates(android, 'Box')

['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [13]:
specific_duplicates(android, 'Google Ads')

['Google Ads', 'BUSINESS', '4.3', '29313', '20M', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'July 30, 2018', '1.12.0', '4.0.3 and up']
['Google Ads', 'BUSINESS', '4.3', '29313', '20M', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'July 30, 2018', '1.12.0', '4.0.3 and up']
['Google Ads', 'BUSINESS', '4.3', '29331', '20M', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'July 30, 2018', '1.12.0', '4.0.3 and up']


In [15]:
specific_duplicates(android, 'Slack')

['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


Several of the duplicates of the apps I checked seem to be the same. However, in 'Slack' I can see that the number of ratings is different. It is fair to assume that, since everything else is the same, the apps with the same name are the same apps with the data collected at different times. 

Thus, the app with more ratings was likely collected last, and will be the row that is kept, while the others are deleted.

In [21]:
expected_clean_length_android = len(android) - 1181
print('Expected length after removal of duplicates:', len(android) - 1181)

Expected length after removal of duplicates: 9659


We now know that the expected length of our dataset after we remove the duplicates is 9.659. We are going to use a dictionary to determine a list of all the apps that should be in our clean dataset. The criterion for adding to the dictionary is: (1) the app is not already in there; or (2) if the app is already in there with a lower rating it will be replaced with the current one. 

If the length of the dictionary is the same as the expected length after removal of duplicates, it was done correctly. 

In [18]:
ratings_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if name not in ratings_max or (name in ratings_max and ratings_max[name] < n_reviews):
        ratings_max[name] = n_reviews
        
print(len(ratings_max))

9659


Using the `ratings_max` dictionary, we will remove the duplicates from the android data. To do this, we will iterate through the android data set, only adding an app to the new clean dataset if it matches the name and ratings in `ratings_max` and has not already been added.

To test that this was successful, we will check the length of the clean data. If it equals the expected length (9,659) then we can be confident it was successful.

In [22]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == ratings_max[name] and name not in already_added:
        already_added.append(name)
        android_clean.append(app)

print('The length of the clean data is:', len(android_clean))
print('Cleaning android data was successful:', expected_clean_length_android == len(android_clean))

The length of the clean data is: 9659
Cleaning android data was successful: True


### iOS Data

Now onto the iOS data. I am going to start in the same way: by examining the number of duplicates, printing some of the names out, and examining the specific apps for trends.

In [25]:
print('Number of duplicate iOS apps:', len(ios_duplicates))

Number of duplicate iOS apps: 0


According to this, there are no duplicates in the iOS data. This is great and allows us to continue to the next step. As mentioned earlier, we will be focusing only on free apps for English-speakers. Let's remove all non-English apps, to start.

## Removing Non-English Apps